In [1]:
import json
# import utils
from cryptography.fernet import Fernet
import os
import random
import array
from lib import gates as gate

In [2]:
# class Gates:
    
#     def NOT(self, tup):
#         try:
#             x = tup[0]
#         except:
#             x = tup
#         if x is 1:
#             return 0
#         return 1
       
#     def OR(self, tup):
#         x, y = tup
#         if x == y and x == 0:
#             return 0
#         return 1
    
#     def AND(self, tup):
#         x, y = tup
#         if x == y and x == 1:
#             return 1
#         return 0
    
#     def XOR(self, tup):
#         x, y = tup
#         if x == y:
#             return 0
#         return 1
    
#     def NOR(self, tup):
#         return self.NOT(self.OR(tup))
    
#     def NAND(self, tup):
#         return self.NOT(self.AND(tup))
   
#     def XNOR(self, tup):
#         return self.NOT(self.XOR(tup))
    
#     def ADD(self,tup):
#         x,y = tup
#         if x is 0:
#             if y is 1:
#                 return 1
#             else:
#                 return 0
#         else:
#             if y is 1:
#                 return 1
#             else:
#                 return 0

In [3]:
def loadall():
    import os
    folderpath = "json"
    for file in os.listdir(folderpath):
        filename = os.path.join(folderpath,file)
        with open(filename) as json_file:
            json_circuits = json.load(json_file)
            
        for json_circuit in json_circuits['circuits']:
            circuit = Circuit(json_circuit)
            circuit.printall()

In [10]:
def load(filename="json/f.smart.json"):
    with open(filename) as json_file:
        json_circuits = json.load(json_file)

    for json_circuit in json_circuits['circuits']:
        circuit = Circuit(json_circuit)
        circuit.printall()

class Circuit:
    def __init__(self,circuitDict, pBitOverride=None):
        """
        Circuit class to be computed by Alice. This means
        Alice knows the p
        """
        self.name = None
        self.alice = None
        self.bob = None
        self.gates = None
        self.out = None
        # processes json.
        self.processCircuit(circuitDict)
        # generate wire encryption keys
        self.generateWireKeys()
        # generate p bits
        self.generatePBits(override=pBitOverride)
        # load logic gates
        self.logic = gate.Gates()

    def processCircuit(self,circuit):
        """
        Loads the json and adds the relevant contents into the
        object.
        """
        for key in circuit.keys():
            setattr(self,key,circuit[key])
        self.raw = circuit
            
    def compute(self,inputs, encrypt=False):
        """
        Computes the gate operations and then returns the output.
        """
        # initiate gates
        gates = Gates()
        # creates stores.
        limit = max([max(self.out),max([i['id'] for i in self.gates])])
        # create array to reference values at.
        store = [0 for i in range(limit+1)]

        # store inputs
        input_ids = self.alice+self.bob if self.bob else self.alice
        for i in input_ids:
            store[i] = inputs.pop(0)
        
        # iterate through the gate operations
        for gate in self.gates:
            # load logic gate
            logic = getattr(gates,gate['type'])
            # load inputs
            if encrypt:
                parameters = tuple([self.xor(store[i],self.p[i]) for i in gate['in']])
            else:
                parameters = tuple([store[i] for i in gate['in']])
            # get output
            result = logic(parameters)
            # store result
            store[gate['id']] = result
        
        if encrypt:
            return [self.xor(store[i],self.p[i]) for i in self.out]
        else:
            return [store[i] for i in self.out]
    
    def generateWireKeys(self):
        wire_count = max([max(self.out),max([i['id'] for i in self.gates])])
        # generate arbitary zeros and ones
        w = [[Fernet.generate_key() for j in range(0,2)] for i in range(wire_count+1)]
        # assign values.
        self.w = w
    
    def generatePBits(self, override=None):
        """
        generates random p bits.
        these are the colouring bits on each wire.
        """
        # each wire w has two random keys (consult p28)
        w = override if override else []
        # get max limit of the gates.
        wire_count = max([max(self.out),max([i['id'] for i in self.gates])])
        
        if override and (len(override) != wire_count):
            print("YOUR P BITS WONT WORK.")
        # generate arbitary zeros and ones
        w = [random.choice([0,1]) for i in range(wire_count+1)]
        self.p = w
    
    def printRow(self, alice, bob, encrypt=False):
        """
        Prints an individual row given inputs from alice and bob.
        alice and bob are lists of binary values i.e [0,0,1]
        """
        if bob:
            inputs = alice + bob
            pr = "Alice"+str(self.alice)+" = " 
            for i in alice:
                pr += str(i) + " "
            pr += "  Bob"+str(self.bob)+" = "
            for i in bob:
                pr += str(i) + " "
            pr += "  Outputs"+str(self.out)+" = "
            for i in self.compute(inputs,encrypt=encrypt):
                pr += str(i) + " "
            print(pr)
        else:
            inputs = alice
            pr = "Alice"+str(self.alice)+" = " 
            for i in alice:
                pr += str(i) + " "
            pr += "  Outputs"+str(self.out)+" = "
            for i in self.compute(inputs,encrypt=encrypt):
                pr += str(i) + " "
            print(pr)
    
    def printall(self, encrypt=False):
        """
        Helper function to handle printing as is specified in the
        coursework notes.
        """
        print(self.name)
        for alice in self.perms(len(self.alice)):
            if self.bob:
                for bob in self.perms(len(self.bob)):
                    self.printRow(alice,bob,encrypt=encrypt)
            else:
                self.printRow(alice,bob=None,encrypt=encrypt)
        print()
        
    def fernetXOR(encryptedToken, p):
        xoredTokenBits = []
        for i in encryptedToken:
            binary = i
            newBinary = [XOR(int(binary[j]),p) for j in range(len(binary))]
            newBinary = ''.join(str(i) for i in newBinary)
            xoredTokenBits.append(newBinary)
        return xoredTokenBits
    
    def encodeFernetXOR(token):
        return ["{0:b}".format(i) for i in token]
    
    def decodeFernetXOR(ku):
        ku = [int(m, 2) for m in ku]
        ku = "".join(map(chr, ku))
        return  ku.encode('utf-8')
    
    def generateGarbledCiruitTables(self,encrypt=True):
        """
        Computes the gate operations and then returns the output.
        """   
        garbledTables = []
        
        # iterate through the gate operations
        for gate in self.gates:
            garbledLogicGateTable = []
            # load logic gate
            logic = getattr(self.logic,gate['type'])
                # generate permutations of inputs.
                
            for potentialInput in self.perms(len(gate['in'])):
                
                encryptedInput = []
                # encrypt the values with w, and then XOR with p.
                for i in range(len(gate['in'])):
                    wire, value = gate['in'][i], potentialInput[i]
                    # xor with w[wire]
                    encryptedValue = self.xor(value,self.w[wire][value])
                    # xor with p[wire]
                    encryptedValue = self.xor(encryptedValue,self.p[wire])
                    encryptedInput.append(encryptedValue)
                    
                parameters = tuple(encryptedInput)  
                # get output
                result = logic(parameters)
                # encrypt output
                encryptedOutput = self.xor(result,self.p[gate['id']])
                # generate dictionary to store.
                entry = {'id': gate['id'], 'inputs' : [], 'outputs':[]}
                for i in range(len(gate['in'])):
                    entry['inputs'].append({
                                  'wire':   gate['in'][i], 
                        'encryptedValue':   parameters[i]
                    })
                # there should be only one output.
                entry['outputs'].append({
                              'wire':   gate['id'], 
                    'encryptedValue':   encryptedOutput
                })
                
                # add table row into our list
                garbledLogicGateTable.append(entry)
            # add our logic gate table to the main table
            garbledTables.append(garbledLogicGateTable)

        return garbledTables
        
        
    def sendToBob(self,aliceInput):
        # garble the table
        garbled = self.generateGarbledCiruitTables()
        # get wire keys
        w = self.w
        # get alice's encrypted bits
        encryptedBits = []
        
        for i in range(len(aliceInput)):
            bit = self.xor(aliceInput[i], self.w[self.alice[i]][aliceInput[i]])
            encryptedBits.append(bit)
            
        # get decryption bit for output wire
        outputDecryptionBit = self.p[self.out[0]]
        
        return {
            'table'  : garbled,
            'w'      : w,
            'aliceIn': encryptedBits,
            'aliceIndex' : self.alice,
            'bobIndex' : self.bob,
            'numberOfIndexes' : max([max(self.out),max([i['id'] for i in self.gates])]),
            'outputDecryption': outputDecryptionBit
        }
        
    @staticmethod
    def perms(n):
        """
        Helper function to generate permutations for binary integers
        based on the length n.
        """
        if not n:
            return
        entries = []
        for i in range(2**n):
            s = bin(i)[2:]
            s = "0" * (n-len(s)) + s
            ent = [int(i) for i in s]
            entries.append(ent)
        return entries
    
    @staticmethod
    def xor(value, key):
        # use once to encrypt
        # twice to decrypt
        if value == key:
            return 0
        return 1

In [5]:
def xor(value, key):
        # use once to encrypt
        # twice to decrypt
        if value == key:
            return 0
        return 1
    
def bobHandler(data,inputs=[1]):
    
    # 'table'  : garbled,
    # 'w'      : w,
    # 'aliceIn': encryptedBits,
    # 'aliceIndex' : self.alice,
    # 'bobIndex' : self.bob,
    # 'numberOfIndexes' : number of indexes..
    # 'outputDecryption': outputDecryptionBit
    
    table, w, aliceIn = data['table'], data['w'], data['aliceIn']
    aliceIndex, bobIndex, outputDecryption = data['aliceIndex'], data['bobIndex'], data['outputDecryption']

    """
    TODO: 
    - test that the bobhandler can get the original output.
    - use encryption instead of xor.
    - build parser for the table.
    - table should be indexed by encryption and p value.
    """
    
    encryptedStore = [0 for i in range(data['numberOfIndexes']+1)]
    
    # encrypt bob's input.
    bobsInput = inputs
    # encrypt bob's input with w.
    bobsEncryptedInputs = [xor(bobsInput[i],w[bobIndex[i]]) for i in range(len(bobIndex))]
    # store bob's input in the store array.
    for i in range(len(bobIndex)):
        encryptedStore[bobIndex[i]] = bobsEncryptedInputs[i]
    # store alices input in the store array.
    for i in range(len(aliceIndex)):
        encryptedStore[aliceIndex[i]] = aliceIn[i]
    
    # TODO: clean this to have a nicer table.
    
    pass
    

In [12]:
folderpath = "json"
for file in os.listdir(folderpath):
    filename = os.path.join(folderpath,file)
    with open(filename) as json_file:
        json_circuits = json.load(json_file)

    for json_circuit in json_circuits['circuits']:
        circuit = Circuit(json_circuit)
        aliceInput = [1]
        toBob = circuit.sendToBob(aliceInput)
        bobHandler(toBob)
#         for i in circuit.generateGarbledCiruitTables():
#             for j in i:
#                 print(j)

#         key = Fernet.generate_key()
#         f = Fernet(key)
#         inpStr = b"my deep dark secret"
#         token = f.encrypt(inpStr)
#         print(token)
#         col = circuit.computeWireColouring(token,1)
#         print(col)
#         colback = circuit.computeWireColouring(col,1)
#         print(colback)
#         print(circuit.w)
#         print(circuit.p)
#         circuit.printall()
#         garble  = GarbledCircuit(circuit)
#         print(circuit.raw)
        break
    break
# loadall()

{'inputs': [{'wire': 1, 'encryptedValue': 0}, {'wire': 2, 'encryptedValue': 1}], 'outputs': [{'wire': 3, 'encryptedValue': 0}], 'id': 3}
{'inputs': [{'wire': 1, 'encryptedValue': 0}, {'wire': 2, 'encryptedValue': 1}], 'outputs': [{'wire': 3, 'encryptedValue': 0}], 'id': 3}
{'inputs': [{'wire': 1, 'encryptedValue': 0}, {'wire': 2, 'encryptedValue': 1}], 'outputs': [{'wire': 3, 'encryptedValue': 0}], 'id': 3}
{'inputs': [{'wire': 1, 'encryptedValue': 0}, {'wire': 2, 'encryptedValue': 1}], 'outputs': [{'wire': 3, 'encryptedValue': 0}], 'id': 3}


In [ ]:

folderpath = "json"
for file in os.listdir(folderpath):
    filename = os.path.join(folderpath,file)
    with open(filename) as json_file:
        json_circuits = json.load(json_file)
    
    for json_circuit in json_circuits['circuits']:
        circuit = Circuit(json_circuit)
#         print(circuit.name)
        if "Smart" in circuit.name:
#             circuit.printRow([0,0], [0,0])
            circuit.printall(encrypt=False)
    #         print(circuit.p)
    #         garble  = GarbledCircuit(circuit)
    #         print(circuit.raw)
            break
        
#     break
    
# loadall()

In [ ]:
class ObliviousTransfer:
    """
    - Alice has m1,m2
    - Bob will learn either m1 or m2 (but not both)
    - Alice gens pub-priv key pairs (Pu1, Pr1), (Pu2,Pr2)
    - Alice sends Bob Pu1, Pu2
    - Bob generates symmetric key K and randomly chooses Pub1
    - Bob computes c=Encrypt_Pu1(K) and sends c to Alice
    - Alice computes Decrypt_Pr1(c)=K [which is the good key]
    - Alice computes Decrypt_Pr2(c)=U [which is the bad key]
    - Alice computes c1=Encrypt_K(m1)
    - Alice computes c2=Encrypt_U(m2)
    - Alice sends c1,c2 to Bob
    - Bob computes D_K(c1) to get the good message m1
    - Bob computes D_K(c2) to get the rubbish message.
    
    """
    def __init__(self):
        self.A = None
        self.B = None
        
    def generatePublicPrivateKeys(self):
        return
    
    @staticmethod
    def primeGen(num_bits):
        return utils.gen_prime(num_bits)

In [ ]:
class GarbledCircuit:
    def __init__(self, circuit):
        self.x = 1
        self.circuit = circuit
        """
        We need to store:
        - garbled tables
        - wire keys
        - encrypted bits for input bits
        - decryption bit for output
        """
        self.generatePBits()
        
    def generatePBits(self, override=None):
        """
        generates random p bits.
        these are the colouring bits on each wire.
        """
        # each wire w has two random keys (consult p28)
        w = override if override else []
        # get max limit of the gates.
        wire_count = max([max(self.circuit.out),max([i['id'] for i in self.circuit.gates])])
        
        if len(override) != wire_count:
            print("YOUR P BITS WONT WORK.")
        # generate arbitary zeros and ones
        w = [random.choice([0,1]) for i in range(wire_count)]
        self.p = w
        
    def encryptGate(self, gate, override=None):
        """
        Generates encryption keys for each wire
        for some given logic gate.
        """
        
        # each wire w has two random keys (consult p28)
        w = override if override else []
  
        # each wire w also has a random 0 or 1 value (key)
        # p(w) which is used to encyrpt the actual values
        # v[w] using an xor. (consult p29)
        
        return
        
    def randomOrder(self, truthTable):
        """sort the truth table in some random order"""
        return
    
    def generateMappingTable(self):
        """To be sent to Bob with the random order"""
        return
    
    def computeGarbledTable(self, gate, keys):
        """
        Alice creates 2 random keys for each wire.
        One key   corresponds to the encryption of value 0
        The other corresponds to the encryption of value 1
        
        you'll need to encrypt all the inputs and outputs.
        - get the # of inputs and outputs for that gate
        - see p21 for the rest of relevant information
        
        - we also have colouring-bits (see p29)
        """
        return
    
    @staticmethod
    def xor(value, key):
        # use once to encrypt
        # twice to decrypt
        if value == key:
            return 0
        return 1

In [ ]:

folderpath = "json"
for file in os.listdir(folderpath):
    filename = os.path.join(folderpath,file)
    with open(filename) as json_file:
        json_circuits = json.load(json_file)

    for json_circuit in json_circuits['circuits']:
        circuit = Circuit(json_circuit)
        circuit.printall()
#         garble  = GarbledCircuit(circuit)
#         print(circuit.raw)
        break
    break
# loadall()

In [ ]:

# also generate encryption on wires
g = Gates()
tup = (1,1)
p_w1 = 1
p_w3 = 0
p_w5 = 0
for i in range(0,2):
    for j in range(0,2):
        tup = (i,j)
        # 
        garbled_tuple = (g.XOR((tup[0], p_w1)), g.XOR((tup[1], p_w3)))

        # test on AND gate
        output = g.AND(garbled_tuple)
        
        powedput = g.XOR((output,p_w5))
        # XOR output wire with it's own p key
        print(i,j,powedput)

In [ ]:

from cryptography.fernet import Fernet
import base64
import ast

key = Fernet.generate_key()
f = Fernet(key)

def XOR(x,y):
        if x == y:
            return 0
        return 1
        
def encodeFernetXOR(token):
    return ["{0:b}".format(i) for i in token]

def fernetXOR(encryptedToken, p=1):
    xoredTokenBits = []
    for i in encryptedToken:
        binary = i
        newBinary = [XOR(int(binary[j]),p) for j in range(len(binary))]
        newBinary = ''.join(str(i) for i in newBinary)
        xoredTokenBits.append(newBinary)
    return xoredTokenBits

def decodeFernetXOR(ku):
    ku = [int(m, 2) for m in ku]
    ku = "".join(map(chr, ku))
    return  ku.encode('utf-8')

def toString(ku):
    return str(ku)

def toEncode(ku):
    return ast.literal_eval(ku)

if __name__ == "__main__":
    inp = '0'
    print()
    print("INPUT    :",inp)
    token = f.encrypt(inp.encode('utf-8'))
    print("ENCRYPTED:",token)


    fernInput = encodeFernetXOR(token)
    # print(binaryFormat)
    ku = fernetXOR(fernetXOR(fernInput))
    # ku = fernetXOR(fernInput)
    kustr = toString(ku)

    # ku = decodeFernetXOR(ku)
    print("KU       :",kustr)

    kude = toEncode(kustr)

    ku = decodeFernetXOR(kude)

    decrp = f.decrypt(ku)

    print("DECRYPTED:",decrp.decode('utf-8'))
    print()
    print()
    print()

    # print(fernetXOR2(fernetXOR2(token)))